In [2]:
from pyspark.sql import SparkSession
import mysql.connector

In [3]:
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

24/06/23 16:16:03 WARN Utils: Your hostname, Ronits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
24/06/23 16:16:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/23 16:16:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### MySQL Connection

In [4]:
conn = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='ronitgupta28',
    database='ExcelAnalysis'
)

cursor = conn.cursor()

### Employee Review Table

In [5]:
query_columns = "SHOW COLUMNS FROM EmployeeReview"
cursor.execute(query_columns)
columns = [column[0] for column in cursor.fetchall()]

query = "SELECT * FROM EmployeeReview"
cursor.execute(query)
data = [row for row in cursor.fetchall()]

df = spark.createDataFrame(data, schema=columns)

df.createOrReplaceTempView('EmployeeReview')

### Employee Survey Table

In [13]:
query_columns = "SHOW COLUMNS FROM EmployeeSurvey"
cursor.execute(query_columns)
columns = [column[0] for column in cursor.fetchall()]

query = "SELECT * FROM EmployeeSurvey"
cursor.execute(query)
data = [row for row in cursor.fetchall()]

df = spark.createDataFrame(data, schema=columns)

df.createOrReplaceTempView('EmployeeSurvey')

## Temporary Tables

### Race

In [40]:
df = spark.sql(
    """WITH RaceCTE AS (
	SELECT DISTINCT Race FROM EmployeeReview
    WHERE Race IS NOT NULL
)
SELECT Race, ROW_NUMBER() OVER (ORDER BY Race) AS RaceID FROM RaceCTE """
)
df.createOrReplaceTempView('RaceMapping')

### Gender

In [41]:
df = spark.sql(
    """WITH GenderCTE AS (
	SELECT DISTINCT  
    CASE 
		WHEN Gender IN ('F', 'Femail', 'Females', 'Femalr', 'Technically female') THEN 'Female'
        WHEN Gender IN ('M', 'male', 'Man') THEN 'Male'
	END AS Gender
        FROM EmployeeReview
)
SELECT Gender, ROW_NUMBER() OVER (ORDER BY Gender) AS GenderID FROM GenderCTE WHERE Gender IS NOT NULL"""
)
df.createOrReplaceTempView('GenderMapping')

### Education Level

In [43]:
df = spark.sql(
    """WITH EducationLevelCTE AS (
	SELECT DISTINCT  
    EducationLevel
        FROM EmployeeReview
        WHERE EducationLevel IS NOT NULL
)
SELECT EducationLevel, ROW_NUMBER() OVER (ORDER BY EducationLevel) AS EducationLevelID FROM EducationLevelCTE"""
)
df.createOrReplaceTempView('EducationLevelMapping')

### EmploymentType

In [44]:
df = spark.sql(
    """WITH EmploymentTypeCTE AS (
	SELECT DISTINCT  
    EmploymentType
        FROM EmployeeReview
        WHERE EmploymentType IS NOT NULL
)
SELECT EmploymentType, ROW_NUMBER() OVER (ORDER BY EmploymentType) AS EmploymentTypeID FROM EmploymentTypeCTE"""
)
df.createOrReplaceTempView('EmploymentTypeMapping')

### ExperienceLevel

In [45]:
df = spark.sql(
    """WITH ExperienceLevelCTE AS (
	SELECT DISTINCT  
    ExperienceLevel
        FROM EmployeeReview
        WHERE ExperienceLevel IS NOT NULL
)
SELECT ExperienceLevel, ROW_NUMBER() OVER (ORDER BY ExperienceLevel) AS ExperienceLevelID FROM ExperienceLevelCTE"""
)
df.createOrReplaceTempView('ExperienceLevelMapping')

### Job Title 

In [46]:
df = spark.sql(
    """WITH JobTitleCTE AS (
	SELECT DISTINCT  
    JobTitle
        FROM EmployeeReview
        WHERE JobTitle IS NOT NULL
)
SELECT JobTitle, ROW_NUMBER() OVER (ORDER BY JobTitle) AS JobTitleID FROM JobTitleCTE"""
)
df.createOrReplaceTempView('JobTitleMapping')

### SeniorityLevel

In [47]:
df = spark.sql(
    """WITH SeniorityLevelCTE AS (
	SELECT DISTINCT  
    SeniorityLevel
        FROM EmployeeReview
        WHERE SeniorityLevel IS NOT NULL
)
SELECT SeniorityLevel, ROW_NUMBER() OVER (ORDER BY SeniorityLevel) AS SeniorityLevelID FROM SeniorityLevelCTE"""
)
df.createOrReplaceTempView('SeniorityLevelMapping')

### Promotions

In [48]:
df = spark.sql(
    """WITH PromotionsCTE AS (
	SELECT DISTINCT  
    Promotions
        FROM EmployeeReview
        WHERE Promotions IS NOT NULL
)
SELECT Promotions, ROW_NUMBER() OVER (ORDER BY Promotions) AS PromotionsID FROM PromotionsCTE"""
)
df.createOrReplaceTempView('PromotionsMapping')

### Industry

In [49]:
df = spark.sql(
    """WITH IndustryCTE AS (
	SELECT DISTINCT  
    Industry
        FROM EmployeeReview
        WHERE Industry IS NOT NULL
)
SELECT Industry, ROW_NUMBER() OVER (ORDER BY Industry) AS IndustryID FROM IndustryCTE"""
)
df.createOrReplaceTempView('IndustryMapping')

### BusinessType

In [50]:
df = spark.sql(
    """WITH BusinessTypeCTE AS (
	SELECT DISTINCT  
    BusinessType
        FROM EmployeeReview
        WHERE BusinessType IS NOT NULL
)
SELECT BusinessType, ROW_NUMBER() OVER (ORDER BY BusinessType) AS BusinessTypeID FROM BusinessTypeCTE"""
)
df.createOrReplaceTempView('BusinessTypeMapping')

### JobPerformance

In [51]:
df = spark.sql(
    """WITH JobPerformanceCTE AS (
	SELECT DISTINCT  
    JobPerformance
        FROM EmployeeReview
        WHERE JobPerformance IS NOT NULL
)
SELECT JobPerformance, ROW_NUMBER() OVER (ORDER BY JobPerformance) AS JobPerformanceID FROM JobPerformanceCTE"""
)
df.createOrReplaceTempView('JobPerformanceMapping')

### IsCompensation

In [53]:
df = spark.sql(
    """WITH IsCompensationCTE AS (
	SELECT DISTINCT  
    IsCompensation
        FROM EmployeeReview
        WHERE IsCompensation IS NOT NULL
)
SELECT IsCompensation, ROW_NUMBER() OVER (ORDER BY IsCompensation) AS IsCompensationID FROM IsCompensationCTE"""
)
df.createOrReplaceTempView('IsCompensationMapping')

### IsGrowth

In [54]:
df = spark.sql(
    """WITH IsGrowthCTE AS (
	SELECT DISTINCT  
    IsGrowth
        FROM EmployeeReview
        WHERE IsGrowth IS NOT NULL
)
SELECT IsGrowth, ROW_NUMBER() OVER (ORDER BY IsGrowth) AS IsGrowthID FROM IsGrowthCTE"""
)
df.createOrReplaceTempView('IsGrowthMapping')

### IsDecisions

In [55]:
df = spark.sql(
    """WITH IsDecisionsCTE AS (
	SELECT DISTINCT  
    IsDecisions
        FROM EmployeeReview
        WHERE IsDecisions IS NOT NULL
)
SELECT IsDecisions, ROW_NUMBER() OVER (ORDER BY IsDecisions) AS IsDecisionsID FROM IsDecisionsCTE"""
)
df.createOrReplaceTempView('IsDecisionsMapping')

### IsGrowthOpportunities

In [56]:
df = spark.sql(
    """WITH IsGrowthOpportunitiesCTE AS (
	SELECT DISTINCT  
    IsGrowthOpportunities
        FROM EmployeeReview
        WHERE IsGrowthOpportunities IS NOT NULL
)
SELECT IsGrowthOpportunities, ROW_NUMBER() OVER (ORDER BY IsGrowthOpportunities) AS IsGrowthOpportunitiesID 
FROM IsGrowthOpportunitiesCTE"""
)
df.createOrReplaceTempView('IsGrowthOpportunitiesMapping')

### BonusStatus

In [57]:
df = spark.sql(
    """WITH BonusStatusCTE AS (
	SELECT DISTINCT  
    BonusStatus
        FROM EmployeeSurvey
        WHERE BonusStatus IS NOT NULL
)
SELECT BonusStatus, ROW_NUMBER() OVER (ORDER BY BonusStatus) AS BonusStatusID FROM BonusStatusCTE"""
)
df.createOrReplaceTempView('BonusStatusMapping')

### EmployeeCount

In [58]:
df = spark.sql(
    """WITH EmployeeCountCTE AS (
	SELECT DISTINCT  
    EmployeeCount
        FROM EmployeeSurvey
        WHERE EmployeeCount IS NOT NULL
)
SELECT EmployeeCount, ROW_NUMBER() OVER (ORDER BY EmployeeCount) AS EmployeeCountID FROM EmployeeCountCTE"""
)
df.createOrReplaceTempView('EmployeeCountMapping')

### IsFairness

In [59]:
df = spark.sql(
    """WITH IsFairnessCTE AS (
	SELECT DISTINCT  
    IsFairness
        FROM EmployeeSurvey
        WHERE IsFairness IS NOT NULL
)
SELECT IsFairness, ROW_NUMBER() OVER (ORDER BY IsFairness) AS IsFairnessID FROM IsFairnessCTE"""
)
df.createOrReplaceTempView('IsFairnessMapping')

In [38]:

df = spark.sql(
    """WITH EmployeeReviewCTE AS (
    SELECT DISTINCT
        `Respondent ID` AS RespondentID,
        to_date(`Start Date`, 'M/d/y H:m') AS StartDate,
        Race,
        CASE
            WHEN Gender IN ('F', 'Femail', 'Females', 'Femalr', 'Technically female') THEN 'Female'
            WHEN Gender IN ('M', 'male', 'Man') THEN 'Male'
            ELSE Gender
        END AS Gender,
        CASE
            WHEN Age LIKE '%-%' THEN
                (CAST(SUBSTRING_INDEX(Age, '-', 1) AS DECIMAL) + CAST(SUBSTRING_INDEX(Age, '-', -1) AS DECIMAL)) / 2
            WHEN Age LIKE '%+%' THEN
                CAST(SUBSTRING_INDEX(Age, '+', 1) AS DECIMAL)
            ELSE
                CAST(Age AS DECIMAL)
        END AS Age,
        EducationLevel,
        EmploymentType,
        ExperienceLevel,
        JobTitle,
        SeniorityLevel,
        Promotions,
        Industry,
        BusinessType,
        JobPerformance,
        IsCompensation,
        IsGrowth,
        IsDecisions,
        IsGrowthOpportunities
    FROM
        EmployeeReview
    WHERE
        `Respondent ID` IS NOT NULL
        AND `Start Date` IS NOT NULL
        AND Race IS NOT NULL
        AND Gender IS NOT NULL
        AND (Age IS NOT NULL AND Age != '')
        AND EducationLevel IS NOT NULL
        AND EmploymentType IS NOT NULL
        AND ExperienceLevel IS NOT NULL
        AND JobTitle IS NOT NULL
        AND SeniorityLevel IS NOT NULL
        AND Promotions IS NOT NULL
        AND Industry IS NOT NULL
        AND BusinessType IS NOT NULL
        AND JobPerformance IS NOT NULL
        AND IsCompensation IS NOT NULL
        AND IsGrowth IS NOT NULL
        AND IsDecisions IS NOT NULL
        AND IsGrowthOpportunities IS NOT NULL
)
SELECT DISTINCT 
FROM EmployeeReviewCTE AS ER LEFT JOIN RaceCTE AS RCTE ON LTRIM(RTRIM(LOWER(ER.Race))) = LTRIM(RTRIM(LOWER(RCTE.Race)))
        """
)
df.show()

24/06/23 17:08:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/23 17:08:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/23 17:08:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/23 17:08:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/23 17:08:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/23 17:08:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/23 1

+--------------------+------+
|                Race|RaceID|
+--------------------+------+
|               Asian|     1|
|               Asian|     2|
|               Asian|     3|
|               Asian|     4|
|Black or African ...|     5|
|Black or African ...|     6|
|Black or African ...|     7|
|Black or African ...|     8|
|Black or African ...|     9|
|Black or African ...|    10|
|Black or African ...|    11|
|Black or African ...|    12|
|Black or African ...|    13|
|Black or African ...|    14|
|Black or African ...|    15|
|Black or African ...|    16|
|Black or African ...|    17|
|Black or African ...|    18|
|Black or African ...|    19|
|Black or African ...|    20|
+--------------------+------+
only showing top 20 rows

